# Topic : Encryption, Decryption & Digital Signature

Group 5

Team Member :

Siek Chhengly
saroun sakura
ly tykea
phin dara





# Import Cryptography Library

In [1]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives import hashes
import hashlib
from cryptography.hazmat.primitives.asymmetric import padding, rsa
from cryptography.exceptions import InvalidSignature

# UserClass (Key Management)

In [2]:
class User:
    def __init__(self, name: str):
        """
        Initialize a User object with a name and generate RSA keys.

        Args:
            name (str): The name of the user. This can be used for identification.

        Generates:
            - A private RSA key.
            - A corresponding public RSA key derived from the private key.
        """
        self.name = name

        # Generate a private RSA key with a public exponent of 65537 and a key size of 2048 bits.
        self.private_key = rsa.generate_private_key(
            public_exponent=65537,
            key_size=2048,
            backend=default_backend()
        )

        # Extract the public key from the generated private key.
        self.public_key = self.private_key.public_key()

    def get_public_key_pem(self) -> str:
        """
        Retrieve the public key in PEM (Privacy-Enhanced Mail) format.

        Returns:
            str: The public key serialized in PEM format, encoded as a UTF-8 string.
        """
        return self.public_key.public_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PublicFormat.SubjectPublicKeyInfo
        ).decode('utf-8')  # Decode the bytes to a UTF-8 string for readability.

    def get_private_key_pem(self) -> str:
        """
        Retrieve the private key in PEM format.

        Returns:
            str: The private key serialized in PEM format, encoded as a UTF-8 string.

        Note:
            The private key is not encrypted in this example, meaning it is stored as plain text.
            For production systems, consider applying encryption to protect sensitive data.
        """
        return self.private_key.private_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PrivateFormat.TraditionalOpenSSL,
            encryption_algorithm=serialization.NoEncryption()  # No encryption applied to the private key.
        ).decode('utf-8')  # Decode the bytes to a UTF-8 string for readability.

    def sign_data(self, data: str) -> bytes:
        """
        Sign a piece of data using the private key.

        Args:
            data (str): The data to be signed. This should be a string.

        Returns:
            bytes: The generated signature in bytes.

        Raises:
            ValueError: If the input data is empty or not a string.
        """
        if not isinstance(data, str):
            raise ValueError("Data must be a string.")
        if not data:
            raise ValueError("Data cannot be empty.")

        return self.private_key.sign(
            data.encode(),  # Convert the data to bytes before signing.
            padding.PSS(
                mgf=padding.MGF1(hashes.SHA256()),  # Mask Generation Function using SHA256.
                salt_length=padding.PSS.MAX_LENGTH  # Use the maximum allowed salt length.
            ),
            hashes.SHA256()  # Hash function for signing.
        )


# Testing the use class

In [3]:
# Create instances of User
alice = User("Alice")
bob = User("Bob")

# Function to print public keys in a formatted way
def print_public_key(user: User):
    """
    Prints the public key of a given user in a formatted manner.

    Args:
        user (User): The User instance whose public key is to be printed.
    """
    print(f"{user.name}'s Public Key (PEM):\n{user.get_public_key_pem()}\n")

# Print Alice's public key
print_public_key(alice)

# Print Bob's public key
print_public_key(bob)

# Compare Alice and Bob's public keys
if alice.get_public_key_pem() == bob.get_public_key_pem():
    print("Alice and Bob have the same public key.")
else:
    print("Alice and Bob have different public keys.")


Alice's Public Key (PEM):
-----BEGIN PUBLIC KEY-----
MIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEAxwgFG8W/O4flhrlzo5+F
P7O9jy/m+j6qwA3XS494I6E0KXd6kyjw+Pc9AUC4LzhoNFNnMNBHJ+fs5biI+Isz
AQIue+RutnKnqlDSrbDP8l9e6QRLHb5+MFM7tT6OYGXoopdisFapm6JQxk1EnNM5
rcOib+utEZ/l4RWRfdhgvKZV5lVaES+2stSVNZqiUmYRxBpzx3plzbfO5tTIVaSS
kWF7DIq5tM4iBxVcqeNEWIIhDmytQUwrpSvoqCQSrMEgUCoydZjcfq3s/qZuNSnI
H9uWo9ZFh95HYGA5xtr3u0Nw+Ip5dIdaaorsVaGNkkP6OuoSUe7wvx7hyMesgC0H
EwIDAQAB
-----END PUBLIC KEY-----


Bob's Public Key (PEM):
-----BEGIN PUBLIC KEY-----
MIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEAykl090P5OoI0eYItcN24
fVlDDdGgglDKUVdcq7sf2TNtQ3D9Vd6hVWNg6T3YVnm/wqn92OFjdrfvarUE4cU0
9RH1F8EdwhTtUuXY+XUQ6mGbWI5B7+BGr1L/8Uhqbcgt+3/omCaXKf4lQnyzafS2
93kHUAZEU4F2Iunhhl5CNK9Llirl32GOuuRhyDciBrvxnLyIEKUMyQtOp3qPM5fT
KDTZdprl5YkuuR0L9Q3EQdOk2ahMlsELYoUzLCaPZmnLd4J8xuXpYiAb8jWSe6kC
VFPUpR9OAKNmn8hjSRKiubiVu3/edZeRxQ28SLnPD5OiGnFp1kd9spAVQ8UG3yfP
KwIDAQAB
-----END PUBLIC KEY-----


Alice and Bob have different public keys.


# Block Class

In [4]:
class Block:
    def __init__(self, block_number, data, previous_hash, creator_name, signature, public_key_pem):
        """
        Initialize a Block object with essential attributes.

        Args:
            block_number (int): Unique identifier for the block.
            data (str): Content or data stored in the block.
            previous_hash (str): Hash of the previous block in the chain.
            creator_name (str): Name of the user who created the block.
            signature (bytes): Digital signature of the block data.
            public_key_pem (str): Public key of the block creator in PEM format.

        Attributes:
            hash (str): SHA-256 hash of the current block, calculated on initialization.
        """
        self.block_number = block_number  # Unique number identifying the block.
        self.data = data  # Main content or data stored in the block.
        self.previous_hash = previous_hash  # Hash of the previous block in the chain.
        self.creator_name = creator_name  # Name of the user who created the block.
        self.signature = signature  # Digital signature of the block data.
        self.public_key_pem = public_key_pem  # Public key of the block creator in PEM format.

        # Calculate the hash of the current block based on its content.
        self.hash = self.calculate_hash()

    def calculate_hash(self):
        """
        Calculate the SHA-256 hash of the block based on its content.

        Returns:
            str: The computed hash as a hexadecimal string.
        """
        # Concatenate the block's key attributes into a single string.
        to_hash = f'{self.block_number}{self.data}{self.previous_hash}{self.creator_name}'
        # Calculate the SHA-256 hash of the concatenated string.
        return hashlib.sha256(to_hash.encode()).hexdigest()

    def verify_signature(self):
        """
        Verify the digital signature of the block's data using the public key.

        Returns:
            bool: True if the signature is valid, False otherwise.
        """
        try:
            # Load the public key from PEM format.
            public_key = serialization.load_pem_public_key(self.public_key_pem.encode())

            # Use the public key to verify the signature.
            public_key.verify(
                self.signature,  # The signature to be verified.
                self.data.encode(),  # Original data that was signed (converted to bytes).
                padding.PSS(
                    mgf=padding.MGF1(hashes.SHA256()),  # Mask Generation Function using SHA256.
                    salt_length=padding.PSS.MAX_LENGTH  # Use the maximum allowed salt length.
                ),
                hashes.SHA256()  # Hashing algorithm used for signature.
            )
            return True  # If verification succeeds, return True.
        except InvalidSignature:
            return False  # If verification fails, return False.
        except Exception as e:
            print(f"Error verifying signature: {e}")
            return False  # Return False for any other exceptions encountered.




# Testing the block class

In [5]:
# Example: Create a block and print its hash
# Assuming 'alice' is an instance of the User class
block = Block(1, "First Block Data", "0", alice.name, alice.sign_data("First Block Data"), alice.get_public_key_pem())
print(f"Block 1 Hash: {block.hash}")
# Verify with function
if(block.verify_signature()):
    print("Signature is valid")
else:
    print("Signature is invalid")

Block 1 Hash: fe99ccc17799ad85294dad33b56912ce46e5ee757613b8f6a72a3d3bb613c3f6
Signature is valid


# Blockchain Class (Managing the Chain)

In [6]:
class Blockchain:
    def __init__(self):
        genesis_block = Block(0, "Genesis Block", "0", "System", None, None)
        self.chain = [genesis_block]

# Adding Blocks to the Blockchain:
    def add_block(self, data, user):
        previous_block = self.chain[-1]
        new_block = Block(
            block_number=len(self.chain),
            data=data,
            previous_hash=previous_block.calculate_hash(),
            creator_name=user.name,
            signature=user.sign_data(data),
            public_key_pem=user.get_public_key_pem()
        )


        #Note: Block can be added if the signature is valid
        if new_block.verify_signature():
            self.chain.append(new_block)
        else:
            print("Invalid signature. Block not added.")

    # Display the detail of each block to verify the block chain is correctly formed
    def print_chain(self):
        for block in self.chain:
            print(f"Block Number: {block.block_number}")
            print(f"Data: {block.data}")
            print(f"Previous Hash: {block.previous_hash}")
            print(f"Creator: {block.creator_name}")
            print(f"Hash: {block.hash}")
            print(f"Signature: {block.signature}")
            print(f"Public Key: {block.public_key_pem}")
            print("-" * 20)

Testing BlockChain Class

In [7]:
# Create a blockchain
blockchain = Blockchain()

# Add three blocks
blockchain.add_block("Block 1 by Alice", alice)
blockchain.add_block("Block 2 by Bob", bob)
blockchain.add_block("Block 3 by Alice", alice)

# Print the blockchain details
blockchain.print_chain()

Block Number: 0
Data: Genesis Block
Previous Hash: 0
Creator: System
Hash: bcc38dbd7d37b444b36c4ad5d342fcd0cc698b83f9e8fa864e639455ed683eab
Signature: None
Public Key: None
--------------------
Block Number: 1
Data: Block 1 by Alice
Previous Hash: bcc38dbd7d37b444b36c4ad5d342fcd0cc698b83f9e8fa864e639455ed683eab
Creator: Alice
Hash: 08b4d9c8b7f1d4c1fc5060d39c130ed5e7b592cdb5f6b5d4c465069e7f4ee676
Signature: b'\x9a\xe6\xf3\x9c&`\xc1c\xa6\x16\xf5/\xcaf\xdc.=P\xcf\xfc\xcc\xc2L]\x9b\xae<\x13\x16\x0b\xb6\xbb\x96\xd4\xd8R\x94W\xb4\xedt\xce\xdc\x9a\x89L:\x96\xce(69G\xfb\xe7*\xff\x02\xe2@`\x07\xe6\x96\x9a\xaa+\xb2\x08\x89c1\x17\x9f\xe7\x1a\x18\xad:\xda[\xf1I!g$\xd3_\x8b\xd84p*\xa4\xee\x08\x14\xa2c\xec\x19\xb6\xa0\xc9\xfe||\x08\xbb^\xa9\x1fsB*\xc9\xbfY\xdc\x97\x02\xcc\xabL\xc7\xe02z\x97\xf4\xd45\xce\xa2\xec\xe0\x03\x00X\xc2#\x18\xf2\x1f\x1b\xc2\xcb\x8dD\x1c\x9cE\x7fp\xc9\xb0\x07\x8e\xc0\x13\x91p\xe95\x16c\xe4\x93\x01\xca\x1a[\x94B.f3\x98\xaf~\xbb\xef\xf1.{\xc3\x1f\xc39\x9e\xe3\xd3\xaa\xc5\xa1\xb